In [1]:
# import modules
from pathlib import Path
import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import make_scorer

from fairlearn.metrics import selection_rate, MetricFrame
from fairlearn.reductions import GridSearch, EqualizedOdds, FalsePositiveRateParity

In [2]:
# set paths
DATAPATH = "./data"
DATAPROCESSEDPATH = "./data_preprocessed"
MODELPATH = "./models"

In [3]:
with open(DATAPROCESSEDPATH + '/data_preprocessed.pkl','rb') as f:
    df = joblib.load(f)

# load trained pipeline
with open(MODELPATH + '/pipeline.pkl','rb') as f:
    pipe = joblib.load(f)    

In [4]:
# show data
df.head()

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,business_or_commercial,loan_amount,Neg_ammortization,interest_only,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,Region,Status
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nob/c,116500,not_neg,not_int,1740.0,EXP,758,CIB,25-34,to_inst,south,1
1,24891,2019,cf,Male,nopre,type2,p1,l1,b/c,206500,not_neg,not_int,4980.0,EQUI,552,EXP,55-64,to_inst,North,1
2,24892,2019,cf,Male,pre,type1,p1,l1,nob/c,406500,neg_amm,not_int,9480.0,EXP,834,CIB,35-44,to_inst,south,0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nob/c,456500,not_neg,not_int,11880.0,EXP,587,CIB,45-54,not_inst,North,0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nob/c,296500,not_neg,not_int,10440.0,CRIF,602,EXP,25-34,not_inst,North,0


In [5]:
# check nas
df.dropna(how='any', axis=0, inplace=True)

In [6]:
# seperate into numerical and categorical features
num_features = list(df.dtypes[(df.dtypes == 'int64') | (df.dtypes == 'float64')].index[2:-1])
cat_features = list(df.dtypes[df.dtypes == 'object'].index)
label = ['Status']
print('-----------------------------------')
print('numeric features: \n', num_features)
print('-----------------------------------')
print('categorical features: \n', cat_features)

-----------------------------------
numeric features: 
 ['loan_amount', 'income', 'Credit_Score']
-----------------------------------
categorical features: 
 ['loan_limit', 'Gender', 'approv_in_adv', 'loan_type', 'loan_purpose', 'Credit_Worthiness', 'business_or_commercial', 'Neg_ammortization', 'interest_only', 'credit_type', 'co-applicant_credit_type', 'age', 'submission_of_application', 'Region']


In [7]:
# Separate into train and test sets
X = df[num_features + cat_features]
y = df[label].values.reshape(-1)
X_train, X_test, y_train, y_test, df_train, df_test = train_test_split(X, y, df, test_size=0.3, stratify=y, random_state=123)

In [8]:
# show trained pipeline
print(pipe)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  [0, 1, 2]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencode',
                                                                   OneHotEncoder(drop='first'))]),
                                                  [3, 4, 5, 6, 7, 8, 9, 10, 11,

In [9]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((94604, 17), (94604,), (40545, 17), (40545,))

# Analysie Model Fairness

In [10]:
S_train = X_train[['Gender','age']]
S_test = X_test[['Gender','age']]
#S_train['age'] = S_train['age'].astype('str')
#S_test['age'] = S_test['age'].astype('str')
S_test.head()

,Gender,age
145003,Sex Not Available,55-64
10348,Joint,35-44
33469,Male,35-44
123975,Sex Not Available,>74
94831,Joint,45-54


In [11]:
def define_age_group(age_range:str):
    age_max = age_range[-2:]
    try:
        age_max = int(age_max)
        if age_max > 50:
            return 'older_50'
        else:
            return 'younger_50'
    except:
        return 'age_unknown'

In [12]:
# too many age subgroups; define only two i.e. over or under 50
S_train['age'] = S_train['age'].apply(lambda x: define_age_group(str(x)))
S_test['age'] = S_test['age'].apply(lambda x: define_age_group(str(x))) 

C:\Users\Sebas\AppData\Local\Temp\ipykernel_15128\398818651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  S_train['age'] = S_train['age'].apply(lambda x: define_age_group(str(x)))
C:\Users\Sebas\AppData\Local\Temp\ipykernel_15128\398818651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  S_test['age'] = S_test['age'].apply(lambda x: define_age_group(str(x)))


In [13]:
S_test.head()

,Gender,age
145003,Sex Not Available,older_50
10348,Joint,younger_50
33469,Male,younger_50
123975,Sex Not Available,older_50
94831,Joint,older_50


In [14]:
# Get metrics by sensitive group from fairlearn
print('\nMetrics by Group:')
metrics = {'selection_rate': selection_rate,
           'accuracy': accuracy_score,
           'recall': recall_score,
           'precision': precision_score,
           'f1': f1_score}

group_metrics = MetricFrame(metrics=metrics,
                             y_true=y_test,
                             y_pred=pipe.predict(X_test),
                             sensitive_features=S_test)

print(group_metrics.by_group)


Metrics by Group:
                             selection_rate  accuracy    recall precision  \
Gender            age                                                       
Female            older_50         0.133651  0.864061  0.495536  0.950071   
                  younger_50       0.129843  0.889118  0.539931  0.987302   
Joint             older_50         0.110263  0.890312  0.501325  0.988251   
                  younger_50       0.096759  0.921354  0.552486  0.985222   
Male              older_50         0.147776   0.84188  0.482359     0.957   
                  younger_50       0.127713  0.880506   0.51715  0.970297   
Sex Not Available older_50         0.149143  0.840877   0.48299  0.950047   
                  younger_50       0.154267  0.850564  0.508233  0.966597   

                                    f1  
Gender            age                   
Female            older_50    0.651345  
                  younger_50  0.698092  
Joint             older_50    0.665202  
     